In [1]:
require 'nn'
require 'rnn'

In [2]:
-- Some useful functions
function genNbyK(n, k, a, b)
    out = torch.LongTensor(n, k)
    for i=1, n do
        for j = 1, k do
            out[i][j] = torch.random(a, b)
        end
    end
    return out
end

function buildModel(model, vocabSize, embeddingSize, metric, adapt, use_cuda)
    -- Small experiments seem to show that the Tanh activations performed better\
    --      than the ReLU for the bow model
    if model == 'bow' then
        print(string.format("Running bag-of-words model to learn %s", metric))
        sentenceLookup = nn.Sequential()
                    :add(nn.LookupTableMaskZero(vocabSize, embeddingSize))
                    :add(nn.Sum(2, 3, true)) -- Not averaging blows up model so keep this true
                    :add(nn.Tanh())
    else
        print(string.format("Running LSTM model to learn %s", metric))
        sentenceLookup = nn.Sequential()
                    :add(nn.LookupTableMaskZero(vocabSize, embeddingSize))
                    :add(nn.SplitTable(2))
                    :add(nn.Sequencer(nn.LSTM(embeddingSize, embeddingSize)))
                    :add(nn.SelectTable(-1))            -- selects last state of the LSTM
                    :add(nn.Linear(embeddingSize, embeddingSize))
                    :add(nn.ReLU())
    end
    local queryLookup = sentenceLookup:clone("weight", "gradWeight") 
    local summaryLookup = sentenceLookup:clone("weight", "gradWeight")
    local pmodule = nn.ParallelTable()
                :add(sentenceLookup)
                :add(queryLookup)
                :add(summaryLookup)

    if model == 'bow' then
        nnmodel = nn.Sequential()
            :add(pmodule)
            :add(nn.JoinTable(2))
            :add(nn.Tanh())
            :add(nn.Linear(embeddingSize * 3, 2))
    else
        nnmodel = nn.Sequential()
            :add(pmodule)
            :add(nn.JoinTable(2))
            :add(nn.ReLU())
            :add(nn.Linear(embeddingSize * 3, 2))
    end

    if adapt then 
        print("Adaptive regularization")
        local logmod = nn.Sequential()
            :add(nn.Linear(embeddingSize * 3, 1))
            :add(nn.LogSigmoid())
            :add(nn.SoftMax())

        local regmod = nn.Sequential()
            :add(nn.Linear(embeddingSize * 3, 2))

        local fullmod = nn.ConcatTable()
            :add(regmod)
            :add(logmod)

        local final = nn.Sequential()
            :add(pmodule)
            :add(nn.JoinTable(2))
            :add(fullmod)

        nnmodel = final
    end

    if use_cuda then
        return nnmodel:cuda()
    end
    return nnmodel
end

function buildPredsummary(summary, chosenactions, inputsentences, select_index)
    if summary == nil then
        summary = torch.zeros(inputsentences:size())
    end
    for i=1, chosenactions:size(1) do
        -- the 2 is for the SELECT index, will have to make this more general later
        if chosenactions[i][select_index] == 1 then
            summary[i]:copy(inputsentences[i])
        end
    end    
    return summary
end

function buildPredsummaryFast(summary, chosenactions, inputsentences, select_index)
    n = inputsentences:size(1)
    k = inputsentences:size(2)
    if summary == nil then
        summary = torch.zeros(inputsentences:size())
    end
    actionmatrix = chosenactions:select(2, select_index):clone():resize(n, 1):view(n, 1):expand(n, k):clone()
--     actionmatrix = chosenactions:select(2, select_index):resize(1, n):view(n, 1):expand(n, k):clone()
    return actionmatrix:cmul(inputsentences:double())
end

In [3]:
-- Setting parameters
n = 10
n_s = 5
k = 7
q = 5
a = 1
b = 100
embDim = 50
SKIP = 1
SELECT = 2

In [4]:
-- Simulating streams and queries
queries = genNbyK(n, q, a, b)

-- Note that the sentences are batched by sentence index so sentences[1] is the first sentence of each article
sentences = {}
for i=1, n_s do
    sentences[i] = genNbyK(n, k, a, b)
end

-- Using this to generate the optimal actions
true_actions = {}
for i=1, n_s do
    ---- Simulating the data
    trueqValues = torch.rand(n, 2)
    
     ---- Generating the max values and getting the indices
    qMaxtrue, qindxtrue = torch.max(trueqValues, 2)
    
    --- I want to select the qindx elements for each row
    true_actions[i] = torch.zeros(n, 2):scatter(2, qindxtrue, torch.ones(trueqValues:size()))
end

In [5]:
local SKIP = 1
local SELECT = 2

-- Simulating streams and queries
queries = genNbyK(n, q, a, b)

-- Note that the sentences are batched by sentence index so sentences[1] is the first sentence of each article
sentences = {}
for i=1, n_s do
    sentences[i] = genNbyK(n, k, a, b)
end

-- Using this to generate the optimal actions
true_actions = {}
for i=1, n_s do 
    ---- Simulating the data
    trueqValues = torch.rand(n, 2)
    
     ---- Generating the max values and getting the indices
    qMaxtrue, qindxtrue = torch.max(trueqValues, 2)
    
    --- I want to select the qindx elements for each row
    true_actions[i] = torch.zeros(n, 2):scatter(2, qindxtrue, torch.ones(trueqValues:size()))
end

model = buildModel('bow', b, embDim, 'f1', false, false)
preds = model:forward({sentences[1], queries, torch.zeros(n, q)})
print("predictions = ")
print(preds)

-- Pulling the best actions
qMax, qindx = torch.max(preds, 2)

-- Here's the fast way to select the optimal action for each query
actions = torch.zeros(n, 2):scatter(2, qindx, torch.ones(preds:size()))

Running bag-of-words model to learn f1	


predictions = 	
-0.2330 -0.0149
-0.0112 -0.0128
-0.0122  0.0438
 0.3804  0.1618
 0.1889  0.0357
 0.2002  0.0523
 0.2425 -0.1758
 0.1067  0.1283
-0.2557 -0.1378
 0.0526 -0.1244
[torch.DoubleTensor of size 10x2]



In [51]:
totalPredsummary = torch.zeros(n, 3 * k)

predsummary1 = buildPredsummaryFast(predsummary, actions, sentences[1], SELECT)
predsummary2 = buildPredsummaryFast(predsummary, actions, sentences[2], SELECT)

--- This is the first round
for i=1, predsummary1:size(1) do
    totalPredsummary[i][{{21-7+1, 21}}]:copy(predsummary1[i])
end

In [52]:
torch.eq(totalPredsummary[1], 0)[2]

1	


In [59]:
s_val =  0
for j = 1, totalPredsummary[1]:size(1) do
    if torch.eq(totalPredsummary[1], 0)[j] == 1 then
        s_val = s_val + 1
    end            
end 

In [60]:
totalPredsummary[1][{{1, 7}}]

 0
 0
 0
 0
 0
 0
 0
[torch.DoubleTensor of size 7]



In [61]:
s_val

14	


In [56]:
indices

In [74]:
i = 1

In [105]:
totalPredsummary[i]

  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
  0
 83
 16
 15
 99
 41
 37
 38
[torch.DoubleTensor of size 21]



In [106]:
totalPredsummary[i][ {{21 - 21+1, 21 - 7} }]

 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
[torch.DoubleTensor of size 14]



In [107]:
i = 1
nws = 21
s_val = 7

In [108]:
-- Sliding from left to right so the arithmetic is weird
totalPredsummary[i][{{ nws - nws + 1, nws - s_val }}]

 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
[torch.DoubleTensor of size 14]



In [73]:
nws = 7
nps = predsummary1:size(1)
for i=1, predsummary1:size(1) do
    if totalPredsummary[2]:sum() == 0 then
        --- This always fills in the beginning for the empty ones
        totalPredsummary[i][{{21-nws+1, 21}}]:copy(predsummary1[i])
    else 
        s_val =  1
        -- This loops finds the position to insert the tensor
        for j = 1, totalPredsummary[i]:size(1) do
            if torch.eq(totalPredsummary[i], 0)[j] == 1 then
                s_val = s_val + 1
            end            
        end 
        totalPredsummary[i][{{ (s_val+nws) - s_val, (s_val+nws) }}]:copy(predsummary1[i])
    end
end

In [63]:
totalPredsummary

Columns 1 to 20
  0   0   0   0   0   0   0   0   0   0   0   0   0   0  83  16  15  99  41  37
  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
  0   0   0   0   0   0   0   0   0   0   0   0   0   0  75  27  43  35  52  60
  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
  0   0   0   0   0   0   0   0   0   0   0   0   0   0  60  48  81  96  76  40
  0   0   0   0   0   0   0   0   0   0   0   0   0   0  34  19  62  60  21  71
  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0

Columns 21 to 21
 38
  0
 76
  0
  0
  0
  0
 94
 23
  0
[torch.DoubleTensor of size 10x21]



In [128]:
totalPredsummary[1], totalPredsummary[2]:sum()

  0
  0
  0
  0
  0
  0
  0
 36
 48
 16
 69
 35
 33
 13
[torch.DoubleTensor of size 14]

0	


In [121]:
totalPredsummary

  0   0   0   0   0   0   0  36  48  16  69  35  33  13
  0   0   0   0   0   0   0   0   0   0   0   0   0   0
  0   0   0   0   0   0   0  44   8  12  28  82  69  99
  0   0   0   0   0   0   0   0   0   0   0   0   0   0
  0   0   0   0   0   0   0   0   0   0   0   0   0   0
  0   0   0   0   0   0   0   0   0   0   0   0   0   0
  0   0   0   0   0   0   0   0   0   0   0   0   0   0
  0   0   0   0   0   0   0   0   0   0   0   0   0   0
  0   0   0   0   0   0   0   0   0   0   0   0   0   0
  0   0   0   0   0   0   0  30  57  60  61  55  29  26
[torch.DoubleTensor of size 10x14]



In [122]:
predsummary1, predsummary2

 36  48  16  69  35  33  13
  0   0   0   0   0   0   0
 44   8  12  28  82  69  99
  0   0   0   0   0   0   0
  0   0   0   0   0   0   0
  0   0   0   0   0   0   0
  0   0   0   0   0   0   0
  0   0   0   0   0   0   0
  0   0   0   0   0   0   0
 30  57  60  61  55  29  26
[torch.DoubleTensor of size 10x7]

  4  54  69  92  40  13  42
  0   0   0   0   0   0   0
 63  80  31  31  96  25  36
  0   0   0   0   0   0   0
  0   0   0   0   0   0   0
  0   0   0   0   0   0   0
  0   0   0   0   0   0   0
  0   0   0   0   0   0   0
  0   0   0   0   0   0   0
 16  66  10  55  41  57  14
[torch.DoubleTensor of size 10x7]



In [ ]:
totalPredsummary

In [110]:
-- Filling in the rightmost part of the data
totalPredsummary[1][{{14-7+1, 14}}]:copy(predsummary1[1])

totalPredsummary[3][{{14-7+1, 14}}]:copy(predsummary1[3])

In [111]:
out, y  = torch.min(totalPredsummary, 2)

In [114]:
totalPredsummary[{{1, 2, 3, 4}, {1, 2}}]

 0  0
 0  0
[torch.DoubleTensor of size 2x2]



In [115]:
totalPredsummary

  0   0   0   0   0   0   0  36  48  16  69  35  33  13
  0   0   0   0   0   0   0   0   0   0   0   0   0   0
  0   0   0   0   0   0   0  44   8  12  28  82  69  99
  0   0   0   0   0   0   0   0   0   0   0   0   0   0
  0   0   0   0   0   0   0   0   0   0   0   0   0   0
  0   0   0   0   0   0   0   0   0   0   0   0   0   0
  0   0   0   0   0   0   0   0   0   0   0   0   0   0
  0   0   0   0   0   0   0   0   0   0   0   0   0   0
  0   0   0   0   0   0   0   0   0   0   0   0   0   0
  0   0   0   0   0   0   0   0   0   0   0   0   0   0
[torch.DoubleTensor of size 10x14]



In [87]:
out, y

 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
[torch.DoubleTensor of size 10x1]

 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
[torch.LongTensor of size 10x1]



In [78]:
predsummary1[1]

 36
 48
 16
 69
 35
 33
 13
[torch.DoubleTensor of size 7]



In [79]:
predsummary1

 36  48  16  69  35  33  13
  0   0   0   0   0   0   0
 44   8  12  28  82  69  99
  0   0   0   0   0   0   0
  0   0   0   0   0   0   0
  0   0   0   0   0   0   0
  0   0   0   0   0   0   0
  0   0   0   0   0   0   0
  0   0   0   0   0   0   0
 30  57  60  61  55  29  26
[torch.DoubleTensor of size 10x7]



In [83]:
totalPredsummary

  0   0   0   0   0   0   0  36  48  16  69  35  33  13
  0   0   0   0   0   0   0   0   0   0   0   0   0   0
  0   0   0   0   0   0   0  44   8  12  28  82  69  99
  0   0   0   0   0   0   0   0   0   0   0   0   0   0
  0   0   0   0   0   0   0   0   0   0   0   0   0   0
  0   0   0   0   0   0   0   0   0   0   0   0   0   0
  0   0   0   0   0   0   0   0   0   0   0   0   0   0
  0   0   0   0   0   0   0   0   0   0   0   0   0   0
  0   0   0   0   0   0   0   0   0   0   0   0   0   0
  0   0   0   0   0   0   0   0   0   0   0   0   0   0
[torch.DoubleTensor of size 10x14]



In [81]:
predsummary1

 36  48  16  69  35  33  13
  0   0   0   0   0   0   0
 44   8  12  28  82  69  99
  0   0   0   0   0   0   0
  0   0   0   0   0   0   0
  0   0   0   0   0   0   0
  0   0   0   0   0   0   0
  0   0   0   0   0   0   0
  0   0   0   0   0   0   0
 30  57  60  61  55  29  26
[torch.DoubleTensor of size 10x7]



In [82]:
totalPredsummary:index(1, torch.LongTensor({1, 2, 3}))

  0   0   0   0   0   0   0  36  48  16  69  35  33  13
  0   0   0   0   0   0   0   0   0   0   0   0   0   0
  0   0   0   0   0   0   0  44   8  12  28  82  69  99
[torch.DoubleTensor of size 3x14]



In [64]:
totalPredsummary[{{1, 2}, {2, 3}}], predsummary1[{{1, 3}, {1, 3 }}]

 0  0
 0  0
[torch.DoubleTensor of size 2x2]

 36  48  16
  0   0   0
 44   8  12
[torch.DoubleTensor of size 3x3]



In [ ]:
function generateNewSummary(predsummaryT, totalPredsummary, actionvec, k)
    for i=1, predsummaryT:size(1) do
        for j = 1, predsummaryT:size(2) do
            if 
            totalPredsummary[i][{k -predsummaryT:size(2)  , k}]
end

In [ ]:
function stackMemory(newinput, memory_hist, memsize, adapt, use_cuda)
    local sentMemory = torch.cat(newinput[1][1]:double(), memory_hist[1][1]:double(), 1)
    local queryMemory = torch.cat(newinput[1][2]:double(), memory_hist[1][2]:double(), 1)
    local sumryMemory = torch.cat(newinput[1][3]:double(), memory_hist[1][3]:double(), 1)
    local rewardMemory = torch.cat(newinput[2]:double(), memory_hist[2]:double(), 1)

    if adapt thena
        regMemory = torch.cat(newinput[4]:double(), memory_hist[4]:double(), 1)
    end 

    if use_cuda then 
        actionMemory = torch.cat(newinput[3]:double(), memory_hist[3]:double(), 1)
    else 
        actionMemory = torch.cat(newinput[3], memory_hist[3], 1)
    end
    --- specifying rows to index 
    if sentMemory:size(1) <= memsize then
        nend = sentMemory:size(1)
        nstart = 1
    else 
        nstart = math.max(memsize - sentMemory:size(1), 1)
        nend = memsize + nstart
    end
    --- Selecting n last data points
    sentMemory = sentMemory[{{nstart, nend}}]
    queryMemory = queryMemory[{{nstart, nend}}]
    sumryMemory = sumryMemory[{{nstart, nend}}]
    rewardMemory = rewardMemory[{{nstart, nend}}]
    actionMemory = actionMemory[{{nstart, nend}}]

    if use_cuda then
        inputMemory = {sentMemory:cuda(), queryMemory:cuda(), sumryMemory:cuda()}
        rewardMemory = rewardMemory:cuda()
        actionMemory = torch.ByteTensor(#actionMemory):copy(actionMemory):cuda()
    end

    inputMemory = {sentMemory, queryMemory, sumryMemory}
    if adapt then
        regMemory = regMemory[{{nstart, nend}}]
        return {inputMemory, rewardMemory, actionMemory, regMemory}
    end 
    return {inputMemory, rewardMemory, actionMemory}
end    